<a href="https://colab.research.google.com/github/Alexjandrito/Alexjandrito/blob/main/Chatbooot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

pip install tflearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 107 kB 5.2 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=c3b1fb7de83fa7aebe9da051762ede20f254562888111381a183faad32831f0e
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn


In [ ]:
#Used in Tensorflow Model
import numpy as np
import tensorflow as tf
import tflearn
import random
import time
import multiprocessing

#Usde to for Contextualisation and Other NLP Tasks.
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

#Other
import json
import pickle
import warnings
warnings.filterwarnings("ignore")

Instructions for updating:
non-resource variables are not supported in the long term
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
print("Procesamiento de datos.....")
with open('intents.json') as json_data:
    intents = json.load(json_data)


words = []
classes = []
documents = []
ignore_words = ['?']
print("Recorre el json para para convertir los datos en clases documentos  y quitar signos .......")
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])    

          

Procesamiento de datos.....
Recorre el json para para convertir los datos en clases documentos  y quitar signos .......


In [ ]:
print("Elimina datos repetidos.......")
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

Elimina datos repetidos.......
121 documents
43 classes ['HR_related_problem', 'Location', 'Weather', 'about', 'appointment status', 'cabin', 'check_leave', 'commission', 'competitors_in_market', 'configuration', 'connect_people', 'cost_lowering', 'customer_satisfaction', 'domain', 'email_id', 'factors_impacting_sale', 'forgot_password', 'gadgets', 'goodbye', 'greeting', 'highest_grossing', 'hours', 'invalid', 'key_customers', 'leave', 'maintainence', 'manufacturing_problems', 'missing_id', 'name', 'noans', 'options', 'order_components', 'order_tracking', 'predict_delay', 'predict_performance', 'project_handling_queries', 'search_department', 'search_person_by_id', 'solve_problems', 'supplier_info', 'thanks', 'turnover', 'version_update']
226 unique stemmed words ["'s", ',', '.', '23a12', '23a31', '32712', '345a23', '431b67', '561a24', '562b78', '@', 'a', 'abc', 'abx', 'accid', 'ai', 'am', 'an', 'analys', 'and', 'anyon', 'appoint', 'appoit', 'ar', 'at', 'awesom', 'bas', 'be', 'been', '

In [ ]:
print("Creando datos del modelo.....")
training = []
output = []
print("Crea una lista de salida vacia.....")
output_empty = [0] * len(classes)

print("Crea el conjunto de datos de entrenamiento y bolsa de palabras..")
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

Creando datos del modelo.....
Crea una lista de salida vacia.....
Crea el conjunto de datos de entrenamiento y bolsa de palabras..


In [ ]:
print("Convertir en array para un procesamiento mas rapido..")
random.shuffle(training)
training = np.array(training)

from tensorflow.python.framework import ops


print("Creacion de listas para entrenamiento.....")
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Construyendo la rede neuronal..")
ops.reset_default_graph()

Convertir en array para un procesamiento mas rapido..
Creacion de listas para entrenamiento.....
Construyendo la rede neuronal..


In [ ]:
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)
print("Entrenando....")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Entrenando....


In [ ]:
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

In [ ]:

start_time = time.perf_counter()

print("Entrenando el modelo de manera normal..")
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
print("Saving the Model.......")
model.save('model.tflearn')

finish_time = time.perf_counter()
print(f"Program finished in {finish_time-start_time} seconds")

Training Step: 15999  | total loss: 0.39202 | time: 0.123s
| Adam | epoch: 1000 | loss: 0.39202 - acc: 0.9728 -- iter: 120/121
Training Step: 16000  | total loss: 0.35312 | time: 0.131s
| Adam | epoch: 1000 | loss: 0.35312 - acc: 0.9755 -- iter: 121/121
--
Saving the Model.......
Program finished in 174.863750366 seconds


In [ ]:
start_time = time.perf_counter()

print("Entrenando el modelo con threads.") 

# Now we will evaluated with more threads
threads=[]
N_THREADS=5
for i in range(N_THREADS):
    threads.append(multiprocessing.Process(model.fit(train_x, train_y, n_epoch=200, batch_size=8, show_metric=True),""))

for thread in threads:
    thread.start()
    
for thread in threads:
    thread.join()

print("Guarda el modelo..")
model.save('model.tflearn')

finish_time = time.perf_counter()
print(f"Program finished in {finish_time-start_time} seconds")

Training Step: 31999  | total loss: 0.01273 | time: 0.093s
| Adam | epoch: 2000 | loss: 0.01273 - acc: 0.9864 -- iter: 120/121
Training Step: 32000  | total loss: 0.01935 | time: 0.101s
| Adam | epoch: 2000 | loss: 0.01935 - acc: 0.9878 -- iter: 121/121
--
Guarda el modelo..
Program finished in 183.065999966 seconds


In [ ]:
print("Se guarda el array..")
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

Se guarda el array..


In [ ]:
print("Abre el entrenamiento....")
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']


with open('intents.json') as json_data:
    intents = json.load(json_data)
    
print("Inicia el modelo......")
# load our saved model
model.load('./model.tflearn')

Abre el entrenamiento....
Inicia el modelo......


In [ ]:
def clean_up_sentence(sentence):
    # It Tokenize or Break it into the constituents parts of Sentense.
    sentence_words = nltk.word_tokenize(sentence)
    # Stemming means to find the root of the word.
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# Return the Array of Bag of Words: True or False and 0 or 1 for each word of bag that exists in the Sentence
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

ERROR_THRESHOLD = 0.25
print("ERROR_THRESHOLD = 0.25")

def classify(sentence):
    # Prediction or To Get the Posibility or Probability from the Model
    results = model.predict([bow(sentence, words)])[0]
    # Exclude those results which are Below Threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # Sorting is Done because heigher Confidence Answer comes first.
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1])) #Tuppl -> Intent and Probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # That Means if Classification is Done then Find the Matching Tag.
    if results:
        # Long Loop to get the Result.
        while results:
            for i in intents['intents']:
                # Tag Finding
                if i['tag'] == results[0][0]:
                    # Random Response from High Order Probabilities
                    return print(random.choice(i['responses']))

            results.pop(0)

ERROR_THRESHOLD = 0.25


In [ ]:
while True:
    input_data = input("You- ")
    answer = response(input_data)
    answer